In [17]:
import faiss
import numpy as np
import json
from typing import List, Dict
import pandas as pd
import torch

## Load FAISS index and mappings:



In [18]:
# Assume you've saved your index and mapping
index = faiss.read_index("/content/judgements_hnsw.index")


In [24]:
import bigframes.pandas as bf

bf.options.bigquery.location = "asia-south2"
bf.options.bigquery.project = "extreme-course-459207-v5"
df = bf.read_gbq("extreme-course-459207-v5.legal_judgement_texts.extracted_judgements")
df = df.to_pandas()

## Get embeddings with legalbert

In [25]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("law-ai/InLegalBERT")
model = AutoModel.from_pretrained("law-ai/InLegalBERT")

In [26]:
def get_embeddings(text):
  encoded_input = tokenizer(text, return_tensors="pt")
  with torch.no_grad():
    model_output = model(**encoded_input)
  return model_output.last_hidden_state[:, 0, :].numpy()


## Retrieve from faiss

In [27]:
df.head(2)

,parties,facts,rulings,legal_citations,id,embedding_text,embedding
0,"[{'name': 'GIRISH REHANI', 'role': 'Petitioner...","The petitioner, Girish Rehani, filed a petitio...",The court granted the petition and extended th...,['Section 29A(4) of the Arbitration and Concil...,f00eddea-a05e-4ea2-aef2-715375bacf00,"Facts: The petitioner, Girish Rehani, filed a ...",[-2.19421834e-01 -1.74186990e-01 3.12896073e-...
1,"[{'name': 'Mohd. Tabrez', 'role': 'Petitioner'...","The petitioner, Mohd. Tabrez, seeks regular ba...",The court dismissed the bail application of Mo...,['Section 439 of the Code of Criminal Procedur...,997ebb55-891f-4fba-aec4-9ef28a63e295,"Facts: The petitioner, Mohd. Tabrez, seeks reg...",[-3.79319996e-01 -6.66519254e-02 7.61400163e-...


In [28]:
def case_summary(parties,facts,rulings,legal_citations):
  return f"""
  parties: {parties}
  facts: {facts}
  rulings: {rulings}
  legal_citations: {legal_citations}
  """

In [46]:
def retrieve_cases(query: str, top_k=5) -> str:
    query_vec = get_embeddings(query)  #
    if len(query_vec.shape) == 1:
        query_vec = np.expand_dims(query_vec.astype("float32"), axis=0)
    elif len(query_vec.shape) == 2 and query_vec.shape[0] != 1:
        raise ValueError("Expected a single query vector")

    if not hasattr(index, "d"):
        raise ValueError("FAISS index is not initialized")

    # Perform search
    distances, indices = index.search(query_vec, top_k)

    # Generate case summaries
    cases = []
    for idx in indices[0]:
        row = df.iloc[idx]
        summary = case_summary(row["parties"], row["facts"], row["rulings"], row["legal_citations"])
        cases.append(summary)

    return "\n".join(cases)


In [48]:
query="Diploma-holding engineers with 10+ years of experience were denied promotion to a Gazetted post requiring a degree. They relied on an old government circular claiming equivalence, but the department rejected it, stating it wasn’t officially notified. Should the court accept the equivalence and allow promotion?"
retrieve_cases(query,5)

"\n  parties: [{'name': 'J.B.BODA SURVEYORS LTD.', 'role': 'Petitioner'}, {'name': 'SURE SH KUMAR SHARMA', 'role': 'Respondent'}]\n  facts: The petitioner, J.B.Boda Surveyors Ltd., is a company engaged in the business of running surveyors. The respondent, Suresh Kumar Sharma, was employed by the petitioner from 1992 until 1998. The petitioner claims that the respondent abandoned his services in 1998. The respondent raised a dispute regarding illegal termination, which was referred to the Labour Court. The Labour Court ruled in favor of the respondent, directing the petitioner to reinstate him with 50% backwages. The petitioner is challenging this award.\n  rulings: The Court dismissed the petition, upholding the Labour Court's award. The Court found no reason to interfere with the Labour Court's decision, stating that the Labour Court rightly adjudicated the dispute and allowed the claim of the respondent workman after giving due consideration to the settled position of law, and thus t

## Agentic flow

In [89]:
from google import genai
from google.genai import types

client = genai.Client(
    vertexai=True,
    project="extreme-course-459207-v5",
    location="us-central1",
)


In [90]:
def gemini_generate(prompt: str, model: str = "publishers/google/models/gemini-2.5-flash-preview-04-17") -> str:

    text = types.Part.from_text(text=prompt)
    contents = [
      types.Content(
        role="user",
        parts=[
          text
        ]
      )
    ]
    generate_content_config = types.GenerateContentConfig(
      temperature = 0.30,
      top_p = 0.95,
      seed = 0,
      max_output_tokens = 8192,
      response_modalities = ["TEXT"],
      safety_settings = [types.SafetySetting(
        category="HARM_CATEGORY_HATE_SPEECH",
        threshold="OFF"
      ),types.SafetySetting(
        category="HARM_CATEGORY_DANGEROUS_CONTENT",
        threshold="OFF"
      ),types.SafetySetting(
        category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
        threshold="OFF"
      ),types.SafetySetting(
        category="HARM_CATEGORY_HARASSMENT",
        threshold="OFF"
      )],
    )

    result=""

    for chunk in client.models.generate_content_stream(
      model = model,
      contents = contents,
      config = generate_content_config,
      ):
      result+=chunk.text+""

    return result


In [91]:
def plaintiff_agent(round_num, case_facts, retrieved, conversation_so_far=None, defendant_last=None):
    if round_num == 1:
        prompt = f"""You are a seasoned plaintiff attorney preparing an opening.
Retrieved Precedent Cases:
{retrieved}

Based on the facts, legal precedents, and prior exchanges,
generate a persuasive and well-reasoned legal argument supporting the plaintiff's position.
Highlight relevant statutes or rulings and strategically counter any opposing points.
"""
    else:
        prompt = f"""You are a seasoned plaintiff attorney preparing a rebuttal argument.

Transcript of the courtroom so far:
{conversation_so_far}

Defendant's last response:
{defendant_last}

Retrieved Precedent Cases:
{retrieved}

Based on the facts, legal precedents, and prior exchanges,
generate a persuasive and well-reasoned legal argument supporting the plaintiff's position.
Highlight relevant statutes or rulings and strategically counter any opposing points.
"""
    return gemini_generate(prompt, model="gemini-2.5-flash-preview-04-17")


In [92]:
def defendant_agent(round_num, plaintiff_arg, retrieved, conversation_so_far=None):
    if round_num == 1:
        prompt = f"""
You are the legal counsel for the defendant in a simulated courtroom debate.
Analyze the latest argument made by the plaintiff:
"{plaintiff_arg}"

Use the following relevant precedent cases to build your rebuttal:
{retrieved}

Your goal is to defend your client convincingly by referencing applicable legal principles,
contrasting case outcomes, and dismantling the plaintiff's position with logic and precedence.

Provide a clear and persuasive counter-argument.
"""
    else:
        prompt = f"""
You are the legal counsel for the defendant in a simulated courtroom debate.
Analyze the latest argument made by the plaintiff:
"{plaintiff_arg}"

Transcript of the courtroom so far:
{conversation_so_far}

Use the following relevant precedent cases to build your rebuttal:
{retrieved}

Your goal is to defend your client convincingly by referencing applicable legal principles,
contrasting case outcomes, and dismantling the plaintiff's position with logic and precedence.

Provide a clear and persuasive counter-argument.
"""
    return gemini_generate(prompt, model="gemini-2.5-flash-preview-04-17")


In [93]:
def judge_agent(transcript, retrieved_cases, case_facts):
    prompt = f"""You are a judicial reasoning engine.
Below is a transcript of a simulated courtroom debate between a plaintiff and a defendant:
{transcript}

You have access to the following precedent cases:
{retrieved_cases}

Analyze the strength, clarity, and legal grounding of both sides.
Consider the relevance and weight of the cited cases.

Provide:
1. A final verdict (e.g., 'Plaintiff wins', 'Defendant wins', 'Dismissed')
2. A reasoning summary that explains your judgment referencing the arguments and precedents.
"""
    return gemini_generate(prompt, model="gemini-2.5-pro-preview-05-06")


In [94]:
def run_debate(query):
    retrieved = retrieve_cases(query, top_k=5)
    conversation = []

    # Round 1
    plaintiff_arg = plaintiff_agent(1, query, retrieved)
    defendant_arg = defendant_agent(1, plaintiff_arg, retrieved)
    conversation.append({"round": 1, "plaintiff": plaintiff_arg, "defendant": defendant_arg})

    # Rounds 2–5
    for round_num in range(2, 6):
        plaintiff_arg = plaintiff_agent(round_num, query, retrieved, conversation, defendant_arg)
        defendant_arg = defendant_agent(round_num, plaintiff_arg, retrieved, conversation)
        conversation.append({
            "round": round_num,
            "plaintiff": plaintiff_arg,
            "defendant": defendant_arg
        })

    # Final Judgment
    judgement = judge_agent(conversation, retrieved, query)
    return conversation, judgement


In [95]:
query="Diploma-holding engineers with 10+ years of experience were denied promotion to a Gazetted post requiring a degree. They relied on an old government circular claiming equivalence, but the department rejected it, stating it wasn’t officially notified. Should the court accept the equivalence and allow promotion?"
run_debate(query)

([{'round': 1,
   'plaintiff': "Okay, here is a draft opening statement for a seasoned plaintiff attorney, incorporating principles from the provided precedents. The specific nature of the plaintiff's case isn't given, so I will frame it as a commercial dispute involving breach of contract and failure to deliver/refund, drawing primarily from the *Volunteer For Village Development* and *Innovative Crafts* principles (breach, failure to perform, entitlement, improper withholding) and potentially touching on *Sudhir Kumar* if the defendant has complicated matters.\n\n---\n\n**Opening Statement**\n\nMay it please the Court, counsel, ladies and gentlemen of the jury.\n\nMy name is [Your Name], and I represent the plaintiff in this case, [Plaintiff's Name/Company]. [Plaintiff's Name/Company] is a [briefly describe plaintiff, e.g., reputable business, hardworking individual] who came to this court seeking justice for a clear wrong committed by the defendant, [Defendant's Name/Company].\n\nTh